In [ ]:
!pip install git+https://github.com/tky823/ssspy.git

In [ ]:
import numpy as np
import scipy.signal as ss
import matplotlib.pyplot as plt
import IPython.display as ipd

In [ ]:
from ssspy.utils.dataset import download_sample_speech_data

In [ ]:
n_sources = 4
max_samples = 10 * 16000
sisec2010_tag = "dev1_female4"
n_fft, hop_length = 4096, 2048

In [ ]:
waveform_src_img = download_sample_speech_data(n_sources=n_sources, sisec2010_tag=sisec2010_tag, max_samples=max_samples, conv=True) # (n_channels, n_sources, n_samples)
waveform_mix = np.sum(waveform_src_img, axis=1)  # (n_channels, n_samples)

In [ ]:
for idx, waveform in enumerate(waveform_mix):
    print("Mixture: {}".format(idx + 1))
    display(ipd.Audio(waveform, rate=16000))
    print()

In [ ]:
from ssspy.bss.ilrma import GaussILRMA

## w/ partitioning function

In [ ]:
np.random.seed(42)

In [ ]:
gauss_ilrma_iss1 = GaussILRMA(
    n_basis=8,
    algorithm_spatial="ISS1", # You can set "ISS" instead of "ISS1".
    domain=2,
    partitioning=True # w/ partitioning function
)
print(gauss_ilrma_iss1)

In [ ]:
_, _, spectrogram_mix = ss.stft(waveform_mix, window="hann", nperseg=n_fft, noverlap=n_fft-hop_length)

In [ ]:
spectrogram_est = gauss_ilrma_iss1(spectrogram_mix, n_iter=100)

In [ ]:
_, waveform_est = ss.istft(spectrogram_est, window="hann", nperseg=n_fft, noverlap=n_fft-hop_length)

In [ ]:
for idx, waveform in enumerate(waveform_est):
    print("Estimated source: {}".format(idx + 1))
    display(ipd.Audio(waveform, rate=16000))
    print()

In [ ]:
plt.figure()
plt.plot(gauss_ilrma_iss1.loss)
plt.show()
plt.close()

## w/o partitioning function

In [ ]:
np.random.seed(42)

In [ ]:
gauss_ilrma_iss1 = GaussILRMA(
    n_basis=2,
    algorithm_spatial="ISS1", # You can set "ISS" instead of "ISS1".
    domain=2,
    partitioning=False, # w/o partitioning function
    normalization="projection_back"
)
print(gauss_ilrma_iss1)

In [ ]:
_, _, spectrogram_mix = ss.stft(waveform_mix, window="hann", nperseg=n_fft, noverlap=n_fft-hop_length)

In [ ]:
spectrogram_est = gauss_ilrma_iss1(spectrogram_mix, n_iter=100)

In [ ]:
_, waveform_est = ss.istft(spectrogram_est, window="hann", nperseg=n_fft, noverlap=n_fft-hop_length)

In [ ]:
for idx, waveform in enumerate(waveform_est):
    print("Estimated source: {}".format(idx + 1))
    display(ipd.Audio(waveform, rate=16000))
    print()

In [ ]:
plt.figure()
plt.plot(gauss_ilrma_iss1.loss)
plt.show()
plt.close()